In [59]:
from selenium.webdriver.chrome import webdriver, options
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.relative_locator import By 
from selenium.webdriver.support.select import Select 
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlsplit
import re
import os
import pandas as pd
from selenium.webdriver.support.color import Color
import time
import traceback


class ProductType:
    SINGLE = 'single'
    MULTI_SIZE = 'multi-size'
    MULTI_COLOR = 'multi-color'
    MULTI_SHADE = 'multi-shade'

def safe_get_element(wd: webdriver.WebDriver, by: By, value:str):
    try:
        element = wd.find_element(by, value)
        return element
    except NoSuchElementException:
        return None
    

def click_element_refresh_stale(wd: webdriver.WebDriver, element: WebElement, locator: tuple[By, str], index = None):
    while True:
        try:
            wd.execute_script("arguments[0].click();", element)
            return element
        except StaleElementReferenceException:
            print('Could not click element. Refreshing...')
            if index is None:
                element = wd.find_element(locator)
            else:
                element = wd.find_elements(locator)[index]
        

def get_variation_images(wd: webdriver.WebDriver, variation_details:dict[str, object]):
    right_arrow = wd.find_element(By.CLASS_NAME, 'athenaProductImageCarousel_rightArrow')
    for i, image in enumerate(wd.find_elements(By.CLASS_NAME, 'athenaProductImageCarousel_image')):
        if i != 0:
            right_arrow = click_element_refresh_stale(wd, right_arrow, ('class name', 'athenaProductImageCarousel_rightArrow'))
        while True:
            try:
                image_src = image.get_attribute('src')
                break
            except StaleElementReferenceException:
                print('image is stale. Refreshing...')
                image = wd.find_elements(By.CLASS_NAME, 'athenaProductImageCarousel_image')[i]
            except Exception:
                print('Unexpected exception while getting image link.')
                image = wd.find_elements(By.CLASS_NAME, 'athenaProductImageCarousel_image')[i]
        variation_details[f'product_image_{i+1}'] = image_src
    return variation_details


def get_variation_misc_details(wd: webdriver.WebDriver, variation_details:dict[str, object], product_id: str):
    variation_details['variant_SKU'] = product_id
    variation_details['product_name'] = wd.find_element(By.CLASS_NAME, 'productName_title').get_attribute('textContent')
    try:
        variation_details['product_rating'] = float(wd.find_element(By.CLASS_NAME, 'productReviewStarsPresentational').get_attribute('aria-label').split(' ')[0])
    except NoSuchElementException:
        variation_details['product_rating'] = None
    try:
        variation_details['number_of_reviews'] = int(wd.find_element(By.CLASS_NAME, 'productReviewStars_numberOfReviews').text.split(' ')[0])
    except NoSuchElementException:
        variation_details['number_of_reviews'] = None
    variation_details['price'] = wd.find_element(By.CLASS_NAME, 'productPrice_price').text.removeprefix('£')
    try:
        wd.find_element(By.CLASS_NAME, 'productAddToBasket-soldOut')
        variation_details['in_stock'] = 'no'
    except NoSuchElementException:
        variation_details['in_stock'] = 'yes'
    return variation_details

def get_multi_size_details(wd: webdriver.WebDriver, product_details: dict[str, object]) -> list[dict[str, object]]:
    variations = []
    ids = [button.get_attribute("data-linked-product-id") for button in wd.find_elements(By.CLASS_NAME, 'athenaProductVariations_box')]
    for product_id in ids:
        button = wd.find_element(By.CSS_SELECTOR, f"button[data-linked-product-id='{product_id}']")
        variation_details = product_details.copy()
        is_selected = safe_get_element(button, By.CLASS_NAME, 'srf-hide')
        if is_selected is None:
            old_price = get_old_price(wd)
            wd.execute_script('arguments[0].click();', button)
            try:
                WebDriverWait(wd, 10).until(EC.staleness_of(old_price))
            except Exception:
                print(f'Could not find old price for url: "{product_details["product_url"]}"')
            button = wd.find_element(By.CSS_SELECTOR, f"button[data-linked-product-id='{product_id}']")
        variation_details = get_variation_misc_details(wd, variation_details, product_id)
        variation_details['size'] = button.text
        variation_details = get_variation_images(wd, variation_details)
        variations.append(variation_details)
    return variations

def get_id_from_url(url:str):
    base_name = os.path.basename(urlsplit(url).path)
    return base_name.split('.')[0].split('-')[0].strip()

def get_old_price(wd: webdriver.WebDriver):
    try:
        return wd.find_element(By.CLASS_NAME, 'productPrice_price')
    except NoSuchElementException:
        return wd.find_element(By.CLASS_NAME, 'productPrice_fromPrice')
    
def rgb_to_hex(rgb: list):
    return '#%02x%02x%02x' % (int(rgb[0]), int(rgb[1]), int(rgb[2]))

def get_multi_color_details(wd: webdriver.WebDriver, product_details: dict[str, object], product_type: str) -> list[dict[str, object]]:
    variations = []
    drop_down_list = wd.find_element(By.CLASS_NAME, 'athenaProductVariations_dropdown')
    select = Select(drop_down_list)
    for option, id in [(x.text, x.get_attribute('value')) for x in select.options if x.text.casefold() != 'Please choose...'.casefold()]:
        variation_details = product_details.copy()
        old_price = get_old_price(wd)
        select = Select(wd.find_element(By.CLASS_NAME, 'athenaProductVariations_dropdown'))
        select.select_by_visible_text(option)
        try:
            WebDriverWait(wd, 10).until(EC.staleness_of(old_price))
        except Exception:
            print(f'Could not find old price for url: "{product_details["product_url"]}"')
        variation_details = get_variation_images(wd, variation_details)
        product_id = get_id_from_url(variation_details['product_image_1'])
        variation_details = get_variation_misc_details(wd, variation_details, product_id)
        if product_type == ProductType.MULTI_COLOR:
            variation_type = 'color'
        elif product_type == ProductType.MULTI_SHADE:
            variation_type = 'shade'
        else:
            raise ValueError(f'Invalid product type: {product_type}')
        variation_details[variation_type] = option
        color = wd.find_element(By.CSS_SELECTOR, f"span[data-value-id='{id}']").value_of_css_property('background-color')
        color = Color.from_string(color).hex
        variation_details[f'{variation_type}_hex'] = color
        variations.append(variation_details)
    return variations


def get_product_details(wd:webdriver.WebDriver, urls: list[str]):
    df = pd.DataFrame()
    for url in urls:
        try:
            wd.get(url)
            product_details = {}
            product_variations = []
            brand_element = wd.find_element(By.CLASS_NAME, 'productBrandLogo_image')
            product_details['product_url'] = url
            product_details['brand_name'] = brand_element.get_attribute('title')
            product_details['brand_logo'] = brand_element.get_attribute('src')
            product_details['primary_SKU'] = get_id_from_url(url)
            for button in wd.find_elements(By.CLASS_NAME, 'productDescription_accordionControl'):
                try:
                    if not button.text:
                        continue
                    button_id = button.get_attribute("id")
                    is_expanded = button.get_attribute('aria-expanded')
                    if is_expanded == 'false':
                        wd.execute_script("arguments[0].click();", button)
                    description_content = wd.find_element(By.ID, button_id.replace('heading', 'content')).text
                    product_details[button.text] = description_content

                except ElementNotInteractableException:
                    print(f'cannot click element with id: {button_id}')
                except Exception  as e:
                    print(f'Unexpected error occurred: {traceback.format_exc()}')
            variation_label = safe_get_element(wd, By.CLASS_NAME, 'athenaProductVariations_dropdownLabel')
            if variation_label is not None:
                variation = variation_label.text.strip()
                if variation.casefold() in color_variation_tags:
                    product_details['product_type'] = ProductType.MULTI_COLOR
                    product_variations = get_multi_color_details(wd, product_details, ProductType.MULTI_COLOR)
                elif variation.casefold() in shade_variation_tags:
                    product_details['product_type'] = ProductType.MULTI_SHADE
                    product_variations = get_multi_color_details(wd, product_details, ProductType.MULTI_SHADE)
                elif variation.casefold() in size_variation_tags:
                    product_details['product_type'] = ProductType.MULTI_SIZE
                    product_variations = get_multi_size_details(wd, product_details)
                else:
                    print(f'Unknown variant type: {variation}')
            else:
                product_details['product_type'] = ProductType.SINGLE
                product_details = get_variation_images(wd, product_details)
                product_id = get_id_from_url(product_details['product_image_1'])
                product_details = get_variation_misc_details(wd, product_details, product_id)
                product_variations = [product_details]
            df = pd.concat([df, pd.DataFrame(product_variations)], ignore_index=True)
        except Exception as e:
            print(f'Unexpected error with trying to fetch data in url "{url}". \n{e}')
    return df

browser_options = options.Options()
browser_options.add_argument('-disable-notifications')
browser_options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies": 2})
# browser_options.add_argument('-headless')

color_variation_tags = [x.casefold() for x in ['colour:', 'color:']]
shade_variation_tags = [x.casefold() for x in ['shade:']]
size_variation_tags = [x.casefold() for x in ['size:']]

PRODUCT_LINKS = ["https://www.cultbeauty.co.uk/westman-atelier-eye-pods/13324061.html?affil=thgppc&countrySelected=Y",
                 "https://www.cultbeauty.co.uk/huda-beauty-lovefest-obsessions-eyeshadow-palette/13899183.html?affil=thgppc&countrySelected=Y",
                 'https://www.cultbeauty.co.uk/hindash-manifesto-lipstick-3.5g-various-shades/13798789.html?affil=thgppc&settingsSaved=Y&shippingcountry=GB&switchcurrency=GBP&countrySelected=Y',
                 'https://www.cultbeauty.co.uk/welleco-nourishing-protein-chocolate-refill/13314044.html?affil=thgppc&countrySelected=Y',
                 'https://www.cultbeauty.co.uk/thank-you-farmer-sun-project-water-sun-cream-spf50/13313863.html?affil=thgppc&countrySelected=Y',
                 'https://www.cultbeauty.co.uk/color-wow-travel-dream-coat-supernatural-spray-50ml/11870457.html',
                 'https://www.cultbeauty.com/huda-beauty-kayali-yum-pistachio-gelato-33-eau-de-parfum-intense-10ml/14272370.html']

# df = get_product_details(PRODUCT_LINKS, browser_options)
# df.head()

In [53]:
with pd.option_context('display.max_columns', None, 'display.max_rows', None):
    display(df.head(100))

,product_url,brand_name,brand_logo,primary_SKU,Why It's Cult,Description,How to Use,Product Details,product_type,product_image_1,product_image_2,product_image_3,product_image_4,variant_SKU,product_name,product_rating,number_of_reviews,price,in_stock,color,color_hex,Full Ingredients List,product_image_5,product_image_6,shade,shade_hex,size
0,https://www.cultbeauty.co.uk/westman-atelier-e...,Westman Atelier,https://static.thcdn.com/design-assets/images/...,13324061,"Formulated with only the finest, skin-loving i...",A luminous eyeshadow trio in two iterations fo...,Gucci created each Eye Pod palette with one cl...,Brand:\nWestman Atelier,multi-color,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,13324062,Westman Atelier Eye Pods,4.20,15,80.00,no,Les Jours - Out of stock,#000000,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.cultbeauty.co.uk/westman-atelier-e...,Westman Atelier,https://static.thcdn.com/design-assets/images/...,13324061,"Formulated with only the finest, skin-loving i...",A luminous eyeshadow trio in two iterations fo...,Gucci created each Eye Pod palette with one cl...,Brand:\nWestman Atelier,multi-color,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,13324063,Westman Atelier Eye Pods,4.20,15,80.00,yes,Les Nuits,#000000,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.cultbeauty.co.uk/huda-beauty-lovef...,Huda Beauty,https://static.thcdn.com/design-assets/images/...,13899183,With a phenomenal fan base (45+ million Instag...,We’re feeling all kinds of groovy and ready to...,Build your base with matte shadows. Pick up th...,Brand:\nHuda Beauty,single,https://static.thcdn.com/images/large/webp//pr...,https://static.thcdn.com/images/large/webp//pr...,https://static.thcdn.com/images/large/webp//pr...,https://static.thcdn.com/images/large/webp//pr...,13899183,Huda Beauty Lovefest Obsessions Eyeshadow Palette,4.92,13,27.00,no,NaN,NaN,"Deep Brown Matte (1) – Mica, Magnesium Myrista...",https://static.thcdn.com/images/large/webp//pr...,https://static.thcdn.com/images/large/webp//pr...,NaN,NaN,NaN
3,https://www.cultbeauty.co.uk/hindash-manifesto...,Hindash,https://static.thcdn.com/design-assets/images/...,13798789,"Taking your beauty out of this world, Hindash’...","Sealing your look with kissable softness, the ...",Pat the lipstick all over lips for a soft tint...,Brand:\nHindash,multi-shade,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,13798791,Hindash Manifesto Lipstick 3.5g (Various Shades),4.77,22,26.00,yes,NaN,NaN,"Octyldodecanol, Polysilicone11, Synthetic Wax,...",https://static.thcdn.com/images/large/original...,NaN,Call me Peaches,#dd8670,NaN
4,https://www.cultbeauty.co.uk/hindash-manifesto...,Hindash,https://static.thcdn.com/design-assets/images/...,13798789,"Taking your beauty out of this world, Hindash’...","Sealing your look with kissable softness, the ...",Pat the lipstick all over lips for a soft tint...,Brand:\nHindash,multi-shade,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,13798790,Hindash Manifesto Lipstick 3.5g (Various Shades),4.77,22,20.80,yes,NaN,NaN,"Octyldodecanol, Polysilicone11, Synthetic Wax,...",https://static.thcdn.com/images/large/original...,https://static.thcdn.com/images/large/original...,Rest in Ross,#ab0033,NaN
5,https://www.cultbeauty.co.uk/welleco-nourishin...,WelleCo,https://static.thcdn.com/design-assets/images/...,13314044,Co-founded by supermodel Elle Macpherson (aka ...,"Nourishing and slimming, Wel

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed

CATEGORY_LINKS = ['https://www.cultbeauty.com/body-wellbeing/tanning-suncare/shop-all.list',
                  'https://www.cultbeauty.com/skin-care.list',
                   'https://www.cultbeauty.com/make-up.list']
# CATEGORY_LINKS = ['https://www.cultbeauty.com/body-wellbeing/tanning-suncare/shop-all.list',
#                   'https://www.cultbeauty.com/skin-care.list',
#                   'https://www.cultbeauty.com/make-up.list',
#                   'https://www.cultbeauty.com/hair-care.list',
#                   'https://www.cultbeauty.com/body-wellbeing.list',
#                   'https://www.cultbeauty.com/fragrance.list',
#                   'https://www.cultbeauty.com/gifts.list',
#                   'https://www.cultbeauty.com/minis.list',
#                   'https://www.cultbeauty.com/sale.list',
#                   'https://www.cultbeauty.com/men.list']

def get_category_links(browser_options: options.Options, url):
    with webdriver.WebDriver(browser_options) as wd:
        page = 1
        wd.get(f'{url}?pageNumber={page}')
        product_details = pd.DataFrame()
        while True:
            product_links = list(set([x.find_element(By.CLASS_NAME, 'productBlock_link').get_attribute('href') for x in wd.find_elements(By.CLASS_NAME, 'productBlock_itemDetails_wrapper')]))
            product_details = pd.concat([product_details, get_product_details(wd, product_links)])
            try:
                next_page_button = wd.find_element(By.CSS_SELECTOR, 'button.responsivePaginationNavigationButton.paginationNavigationButtonNext')
            except NoSuchElementException:
                print(f'Could not find next button in: "{url}. Page: {page}"')
                return product_details
            if next_page_button.get_attribute('disabled') == 'true':
                break
            page += 1
        return product_details

def main():
    all_links = pd.DataFrame()
    with ProcessPoolExecutor(max_workers = 3) as executor:
        results = executor.map(get_category_links, [browser_options for link in CATEGORY_LINKS],CATEGORY_LINKS)
    for result in results:
        all_links = pd.concat([all_links, result])
    print(all_links.shape)
    all_links.head()

if __name__ == '__main__':
    main()


Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


Unexpected error with trying to fetch data in url "https://www.cultbeauty.com/darling-the-travel-kit/14659553.html". 
Message: no such element: Unable to locate element: {"method":"css selector","selector":".productBrandLogo_image"}
  (Session info: chrome=115.0.5790.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55875f0784e3 <unknown>
#1 0x55875eda7c76 <unknown>
#2 0x55875ede3c96 <unknown>
#3 0x55875ede3dc1 <unknown>
#4 0x55875ee1d7f4 <unknown>
#5 0x55875ee0303d <unknown>
#6 0x55875ee1b30e <unknown>
#7 0x55875ee02de3 <unknown>
#8 0x55875edd82dd <unknown>
#9 0x55875edd934e <unknown>
#10 0x55875f0383e4 <unknown>
#11 0x55875f03c3d7 <unknown>
#12 0x55875f046b20 <unknown>
#13 0x55875f03d023 <unknown>
#14 0x55875f00b1aa <unknown>
#15 0x55875f0616b8 <unknown>
#16 0x55875f061847 <unknown>
#17 0x55875f071243 <unknown>
#18 0x7f67c788f18a <unknown>

image is stale. Refreshing.

KeyboardInterrupt: 

Unexpected error with trying to fetch data in url "https://www.cultbeauty.com/glow-recipe-x-barbie-kit/14853476.html". 
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.110)
Stacktrace:
#0 0x55875f0784e3 <unknown>
#1 0x55875eda7c76 <unknown>
#2 0x55875ed81c6c <unknown>
#3 0x55875ee07f8f <unknown>
#4 0x55875ee1ad66 <unknown>
#5 0x55875ee02de3 <unknown>
#6 0x55875edd82dd <unknown>
#7 0x55875edd934e <unknown>
#8 0x55875f0383e4 <unknown>
#9 0x55875f03c3d7 <unknown>
#10 0x55875f046b20 <unknown>
#11 0x55875f03d023 <unknown>
#12 0x55875f00b1aa <unknown>
#13 0x55875f0616b8 <unknown>
#14 0x55875f061847 <unknown>
#15 0x55875f071243 <unknown>
#16 0x7f67c788f18a <unknown>

Unexpected error with trying to fetch data in url "https://www.cultbeauty.com/dr-dennis-gross-skincare-drx-spectralite-eyecare-pro/12553406.html". 
Message: no such window: target window already closed
from unknown error: web view not found
  (Session

In [61]:
import pandas as pd

df = pd.read_excel('./test_cult_beauty.xlsx')

In [64]:
df.shape

(797, 33)

In [81]:
with pd.option_context("display.max_columns", None, 'max_colwidth', None):
    display(df.loc[df['variant_SKU'] == 'default'])

,product_url,brand_name,brand_logo,primary_SKU,Why It's Cult,Description,How to Use,Full Ingredients List,Product Details,product_type,product_image_1,product_image_2,product_image_3,product_image_4,product_image_5,variant_SKU,product_name,product_rating,number_of_reviews,price,in_stock,size,product_image_6,product_image_7,product_image_8,product_image_9,Cult Conscious,product_image_10,product_image_11,shade,shade_hex,color,color_hex
10,https://www.cultbeauty.com/supergoop-glowscreen-spf-30/13350013.html,Supergoop!,https://static.thcdn.com/design-assets/images/logos/shared-brands/colour/supergoop!.gif,13350013,"Pioneers of the sunscreen sphere, Supergoop! has amassed legions of fans for their notorious {Unseen Sunscreen}, and the Glowscreen SPF 30 is its dewier, glowier twin set to protect and prime complexions under the duress of UVA and UVB rays with chemical actives that actually work. Giving you peace of mind when the sunshine hits, youll find this formula also plays well under make up to give a supple, bouncy and radiant finish that won't be mistaken for shine or sweat.","Just like the brands infamous and industry-adored {Unseen Sunscreen}, the Glowscreen SPF 30 primes and protects gripping onto make up while shielding you from UVA and UVB rays but with a dewy, radiant finish rather than Unseen Sunscreens matte and velvety one.\nGiving you a lit-from-within glow (that never looks oily or greasy) this SPF and primer is suitable for all skin types working to flood skin with moisture via a unique blend of hyaluronic acid with B5 that leaves skin looking supple, plump and dewy (without the need for glitter!). Your soon-to-be glow-to, the formula uses niacinamide to soothe, soften and tighten pores, while sea lavender offers powerful antioxidant protection and cocoa peptides guard skin from the damaging effects of free radicals. Reef-safe and cruelty-free, this sheer, skin-perfecting formula creates a deliciously dewy layer for later make up application and won’t leave a chalky cast.",Apply generously and evenly as the last step in your skincare routine and before make up.,"Aqua, Octocrylene, Butyloctyl Salicylate, Ethylhexyl Salicylate, Propanediol, Glycerin, Butyl Methoxydibenzoylmethane, C12-15 Alkyl Benzoate, Glyceryl Stearate Citrate, Niacinamide, Polymethylsilsesquioxane, CI 77163, Mica, CI 77891, Caprylic/Capric Triglyceride, Cetyl Phosphate, Diisopropyl Sebacate, Glyceryl Stearate, Isodecyl Neopentanoate, Isododecane, Lauryl Lactate, 1,2-Hexanediol, Acrylates/C10-30 Alkyl Acrylate Crosspolymer, Arginine, Butylene Glycol, Caprylyl Glycol, Coco-Caprylate, Ethylhexyl Hydroxystearate, Ferulic Acid, Helianthus Annuus Seed Oil, Hydroxyacetophenone, Leuconostoc/Radish Root Ferment Filtrate, Limonium Gerberi Extract, Pantothenic Acid, Phospholipids, Sodium Hyaluronate, Theobroma Cacao Seed Extract, Tocopherol, Trisodium Ethylenediamine Disuccinate, Chlorphenesin, CI 77491\nNOTE: cocoa peptides (Theobroma Cacao Seed Extract, etc.) protect skin against the effects of blue light by targeting free radicals.",Brand:\nSupergoop!,multi-size,https://static.thcdn.com/images/large/original//productimg/1600/1600/13350013-1804898318889201.jpg,https://static.thcdn.com/images/large/original//productimg/1600/1600/13350013-1744898444116890.jpg,NaN,NaN,NaN,default,Supergoop! Glowscreen SPF 30,4.32,356.0,19.20€,yes,15ml,NaN,NaN,NaN,NaN,"Transparency lies at the heart of our philosophy which is why we have partnered with tech-platform – Provenance – to cut through the industry ‘noise’ and equip you with key facts that matter. A 'proof point' with a green tick means a third party has verified the accuracy of the statement whereas no green tick means that there isn't independent confirmation (yet!), but that the brand has still supplied substantiating evidence (which you can view yourself). Discover the proof points for this product below.",NaN,NaN,NaN,NaN,NaN,NaN
11,https://www.cultbeauty.com/supergoop-glowscreen-spf-30/13350013.html,Supergoop!,h

In [73]:
with pd.option_context("display.max_rows", None):
    display(df['variant_SKU'].value_counts(sort=True, ascending=False))

variant_SKU
default     28
12920744     3
12920751     3
12920734     2
11174179     2
13323809     2
14293490     2
13323808     2
13323807     2
14293491     2
12920738     2
12920737     2
12920735     2
13528408     2
12920733     2
11174178     2
11331364     2
12753548     2
12753544     2
13323366     2
13323360     2
13323356     2
11560495     2
13886862     2
12920736     2
13524172     2
13187076     2
12243648     2
13951203     2
13951194     2
13951195     2
13456682     2
14228307     2
13943888     2
14228298     2
13613319     2
14228325     2
14228334     2
13505148     2
13505155     2
13505154     2
13496980     2
13496988     2
13496994     2
13941291     2
12243647     2
11363395     2
13886861     2
13323810     2
12920743     2
12920741     2
13886860     2
13886863     2
13886864     2
13886865     2
13941313     2
12436415     2
12920749     2
13886859     2
12920748     2
12753334     2
11560494     2
13886866     2
12920752     2
12920742     2
10789166     

In [28]:
import pandas as pd

df = pd.read_excel('./test_cult_beauty_with_duplicates.xlsx')

In [29]:
df.drop_duplicates('variant_SKU', inplace=True, ignore_index=True)

In [30]:
import os
from urllib.parse import urlsplit
def get_id_from_url(url:str):
    base_name = os.path.basename(urlsplit(url).path)
    return base_name.split('.')[0].split('-')[0].strip()
df['primary_SKU'] = df['primary_SKU'].transform(get_id_from_url)

In [31]:
def create_serialized_sku(group:pd.Series, mask):
    count = 2
    serialized_skus = []
    for idx, row in group.items():
        if mask[idx]:
            serialized_skus.append((f"{row}-1", pd.NA))
        else:
            serialized_skus.append((f"{row}-{count}", row))
            count += 1
    return pd.Series(serialized_skus, index=group.index)

mask = df['primary_SKU'] == df['variant_SKU']
transform = df.groupby('primary_SKU')['primary_SKU'].transform(create_serialized_sku, mask)
df[['serialized_primary_SKU', 'is_variant_of']] = pd.DataFrame(transform.to_list(), columns=['serialized_primary_SKU', 'is_variant_of']
                                                                , index=transform.index)

In [32]:
import re
df['price'] = df['price'].transform(lambda x: re.sub(r'[^\d.]', '', x))

In [33]:
df.shape

(14044, 46)

In [48]:
def order_serialized_columns(columns: list[str], regex = r'_(\d+)'):
    ordered_columns = []
    groups = {}
    for i, column in enumerate(columns):
        index = re.search(regex, column)
        if index is None or index.group(1) is None:
            ordered_columns.append(column)
            continue
        index = int(index.group(1))
        group_name = re.sub(regex, '', column)
        if group_name not in groups:
            groups[group_name] = {'starting_index': i, 'names': [{'index':index, 'name':column}]}
        else:
            groups[group_name]['names'].append({'index':index, 'name':column})
            if i < groups[group_name]['starting_index']:
                groups[group_name]['starting_index'] = i
    for group in groups.values():
        group['names'] = sorted(group['names'], key=lambda d: d['index'], reverse=True) 

        for name in group['names']:
            ordered_columns.insert(group['starting_index'], name['name'])

    return ordered_columns
df = df.reindex(order_serialized_columns(df.columns), axis=1)

In [49]:
df.dropna(axis=1, how='all', inplace=True)

In [51]:
df.count()

product_url               14044
brand_name                12912
brand_logo                12912
primary_SKU               14044
Why It's Cult             13672
Description               14010
How to Use                13981
Full Ingredients List     13218
Product Details           14038
product_type              14044
option                      403
product_image_1           14044
product_image_2           11890
product_image_3            9743
product_image_4            6997
product_image_5            4807
product_image_6            2629
product_image_7            1501
product_image_8             812
product_image_9             442
product_image_10            213
product_image_11            122
product_image_12             47
product_image_13             30
product_image_14             16
product_image_15              7
product_image_16              3
product_image_17              2
product_image_18              2
product_image_19              2
variant_SKU               14044
product_

In [50]:

df.to_excel('./test_cult_beauty_without_duplicates.xlsx', index=False)